In [1]:
#update libs
!pip install --upgrade pandas
!pip install --upgrade scikit-learn

Requirement already up-to-date: pandas in c:\anaconda\lib\site-packages (0.23.4)
  Found existing installation: scikit-learn 0.20.1
    Uninstalling scikit-learn-0.20.1:
      Successfully uninstalled scikit-learn-0.20.1


In [2]:
#import common libs
import numpy as np
import pandas as pd
import scipy.stats as st
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)
from scipy import interp

In [100]:
#import special libs
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, Imputer,OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PowerTransformer,QuantileTransformer
from sklearn.preprocessing import label_binarize,MultiLabelBinarizer
from sklearn.pipeline import make_pipeline,Pipeline

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from itertools import cycle

from sklearn.multiclass import OneVsRestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split 
from sklearn.model_selection import KFold,RandomizedSearchCV, cross_val_score

from sklearn.metrics import confusion_matrix, recall_score, brier_score_loss, roc_curve, auc
from sklearn.metrics import f1_score, make_scorer, roc_auc_score, precision_score
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA, TruncatedSVD,IncrementalPCA

import glob

from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.calibration import CalibratedClassifierCV, calibration_curve


%matplotlib inline

In [116]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer # one more stemmer? or lemmatisation
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords

seed = 321

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chief\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chief\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Загрузим датасет с классифицируемыми текстами.

In [102]:
from sklearn.datasets import fetch_20newsgroups

Итак, задача данной работы:



#### EDA.

Посмотрим на данные, для этого:
- скачаем датасет целиком
- посмотрим на категории текстов
- распределение документов по категориям
- посмотрим на примеры данных

In [103]:
# здесь shuffle = true, т.к. мы собираемся использовать SGDClassifier; Naive Bayes - они базируются на принципе iid данных 
#и чтобы не получить смещенную оценку
newsgroups_test = fetch_20newsgroups(subset='test', random_state=seed, shuffle=True, remove = ('headers', 'footers', 'quotes'))
newsgroups_train = fetch_20newsgroups(subset='train', random_state=seed, shuffle=True, remove = ('headers', 'footers', 'quotes'))
newsgroups_all = fetch_20newsgroups(subset='all', random_state=seed, shuffle=True, remove = ('headers', 'footers', 'quotes'))

from pprint import pprint
pprint(list(newsgroups_all.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [104]:
print("%d documents" % len(newsgroups_all.data))
print("%d categories" % len(newsgroups_all.target_names))

18846 documents
20 categories


Посмотрим на образец данных.

In [105]:
newsgroups_all.data[3]

"You can't call time when there's a play in progress.\n\nRyan Robbins\nPenobscot Hall\nUniversity of Maine"

In [106]:
newsgroups_all.target_names[newsgroups_train.target[3]]

'soc.religion.christian'

#### Data preprocessing.

Предобработаем данные, для этого:
- приведем все слова к нижнему регистру.
- удалим знаки препинания.
- заменим представление чисел в виде цифр на текстовые аналоги.
- отсечем стоп слова (уберем символы и комбинации, которые не составляют смысловой нагрузки).
- проведем лемматизацию (приведение слова к начальной форме с помощью словаря).
- выделим n-грамма (последовательности из нескольких слов, рассматриваемых как одно целое).

In [117]:
cachedStopWords = stopwords.words('english')
##stopwords = 
def tokenize(text):
    min_length = 3
    words = map(lambda word: word.lower(), word_tokenize(text))
    words = [word for word in words
                  if word not in cachedStopWords]
    tokens = (list(map(lambda token: PorterStemmer().stem(token),
                  words)));
    p = re.compile('[a-zA-Z]+');
    filtered_tokens = list(filter(lambda token:
                  p.match(token) and len(token)>=min_length,
         tokens))
    return filtered_tokens

In [120]:
newsgroups_all.data[3] = tokenize(newsgroups_all.data[3])

In [128]:
newsgroups_all.data[3]

["n't",
 'call',
 'time',
 'play',
 'progress',
 'ryan',
 'robbin',
 'penobscot',
 'hall',
 'univers',
 'main']